In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine
import pandas as pd
import random
from string import ascii_letters
import numpy as np

In [4]:
import sqlalchemy
sqlalchemy.__version__

'2.0.20'

In [5]:
# Function to extract table to a pandas DataFrame
def extract_table_to_pandas(tablename, db_engine):
    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine)

db_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('root', '14842151d', 
                                                          'localhost', 'elearning'))

In [6]:
def print_user_comparison(user1, user2, user3):
    print("Course id overlap between users:")
    print("================================")
    print("User 1 and User 2 overlap: {}".format(
    set(user1.course_id) & set(user2.course_id)
    ))
    print("User 1 and User 3 overlap: {}".format(
    set(user1.course_id) & set(user3.course_id)
    ))
    print("User 2 and User 3 overlap: {}".format(
    set(user2.course_id) & set(user3.course_id)
    ))

In [7]:
# Get user with id 4387
user1 = pd.read_sql("SELECT * FROM rating where user_id=4387", db_engine)

# Get user with id 18163
user2 = pd.read_sql("SELECT * FROM rating where user_id=18163", db_engine)

# Get user with id 8770
user3 = pd.read_sql("SELECT * FROM rating where user_id=8770", db_engine)

# Use the helper function to compare the 3 users
print_user_comparison(user1, user2, user3)

Course id overlap between users:
User 1 and User 2 overlap: {32, 96, 36, 6, 7, 44, 95}
User 1 and User 3 overlap: set()
User 2 and User 3 overlap: set()


In [8]:
def extract_rating_data(db_engine):
    return pd.read_sql("SELECT * FROM rating", db_engine)

def extract_course_data(db_engine):
    return pd.read_sql("SELECT * FROM courses", db_engine)

In [9]:
ratings = extract_rating_data(db_engine)

course_data = extract_course_data(db_engine)

In [10]:
ratings.head()

,user_id,course_id,rating
0,1,6,4
1,1,36,5
2,1,37,5
3,1,45,5
4,1,50,5


In [11]:
course_data.head()

,course_id,title,description,programming_language
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets
2,3,Intermediate R,The intermediate R course is the logical next ...,r
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r


In [12]:
ratings.user_id.nunique()

38174

In [13]:
course_data.shape

(100, 4)

In [14]:
# Print out the number of missing values per column
print(course_data.isnull().sum())

course_id               0
title                   0
description             0
programming_language    3
dtype: int64


In [15]:
# The transformation should fill in the missing values
def transform_fill_programming_language(course_data):
    imputed = course_data.fillna({"programming_language": "R"})
    return imputed

transformed = transform_fill_programming_language(course_data)

# Print out the number of missing values per column of transformed
print(transformed.isnull().sum())

course_id               0
title                   0
description             0
programming_language    0
dtype: int64


In [16]:
# transformation function
def transform_avg_rating(rating_data):
    # Group by course_id and extract average rating per course
    avg_rating = rating_data.groupby('course_id').rating.mean()
    # Return sorted average ratings per course
    sort_rating = avg_rating.sort_values(ascending=False).reset_index()
    return sort_rating

In [17]:
# Use transform_avg_rating on the extracted data and print results
avg_course_ratings = transform_avg_rating(ratings)
print(avg_course_ratings) 

    course_id    rating
0          46  4.800000
1          23  4.800000
2          96  4.692765
3          56  4.661765
4          24  4.653061
..        ...       ...
94         54  4.238095
95         92  4.222222
96         29  4.208333
97         17  4.147059
98         42  4.107570

[99 rows x 2 columns]


In [18]:
 def get_courses_to_recommend(ratings, course_data):
    user_list = ratings.user_id.unique()

    course_set = set(course_data.course_id.unique())

    non_enrolled_users = []

    for user in user_list:
        rated_course_set = set(ratings[ratings.user_id == user].course_id.unique())
        non_enrolled_courses = course_set - rated_course_set
        non_enrolled_users = non_enrolled_users + [(user, cid) for cid in non_enrolled_courses]
        
    non_enrolled_courses_df = pd.DataFrame(non_enrolled_users, columns = ['user_id', 'course_id'])
    return non_enrolled_courses_df

 courses_to_recommend = get_courses_to_recommend(ratings, course_data)
 courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
 courses_to_recommend.sample(n=500).to_csv('courses_to_recommend_small.csv', index = False)

In [19]:
course_data.shape

(100, 4)

In [20]:
temp = pd.read_csv('courses_to_recommend.csv')
temp.sample(n=2000).to_csv('courses_to_recommend_small.csv', index = False)

In [21]:
temp.shape

(3758228, 2)

In [22]:
courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')

courses_to_recommend.head()

,user_id,course_id
0,13863,27
1,10960,7
2,25775,82
3,2684,9
4,14582,67


In [23]:
courses_to_recommend.groupby('user_id').count()

,course_id
user_id,
9,1
25,1
36,1
69,1
100,1
...,...
38098,1
38111,1
38133,1


In [24]:
# Complete the transformation function
def transform_recommendations(avg_course_ratings, courses_to_recommend):
    # Merge both DataFrames
    merged = courses_to_recommend.merge(avg_course_ratings) 
    
    # Sort values by rating and group by user_id
    grouped = merged.sort_values("rating", ascending=False).groupby("user_id")
    
    # Produce the top 3 values and sort by user_id
    recommendations = grouped.head(5).sort_values("user_id").reset_index()
    
    final_recommendations = recommendations[["user_id", "course_id","rating"]]
    
    # Return final recommendations
    return final_recommendations


# Use the function with the predefined DataFrame objects
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [25]:
 def get_courses_to_recommend(programming_languages, course_data):
    df = pd.DataFrame(data=list(range(len(course_data))), columns=["id"])
    Levels = np.random.choice(['Beginner', 'Intermediate', 'Advanced'], size=len(df))
    df['Levels'] = Levels
    
    courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
    courses_to_recommend.sample(n=500).to_csv('courses_to_recommend_small.csv', index = False)
    courses_to_recommend = get_courses_to_recommend(programming_languages, course_data)

In [26]:
recommendations.head()

,user_id,course_id,rating
0,9,66,4.540984
1,25,31,4.610209
2,36,94,4.529412
3,69,59,4.535211
4,100,28,4.578947


In [27]:
# Merge both DataFrames
merged = courses_to_recommend.merge(avg_course_ratings) 

merged.head()

,user_id,course_id,rating
0,13863,27,4.428571
1,5396,27,4.428571
2,29518,27,4.428571
3,3374,27,4.428571
4,8818,27,4.428571


In [28]:
merged.sort_values("rating", ascending=False).head()

,user_id,course_id,rating
331,36032,23,4.8
346,600,23,4.8
1169,33243,46,4.8
1168,28999,46,4.8
1167,802,46,4.8


In [29]:
merged[merged.user_id == 30251]

,user_id,course_id,rating


In [30]:
# Sort values by rating and group by user_id
grouped = merged.sort_values("rating", ascending=False).groupby("user_id")


# Produce the top 3 values and sort by user_id
recommendations = grouped.head(3).sort_values("user_id")

recommendations.head()

,user_id,course_id,rating
1144,9,66,4.540984
669,25,31,4.610209
164,36,94,4.529412
737,69,59,4.535211
1794,100,28,4.578947


In [31]:
recommendations[recommendations.user_id == 16456]

,user_id,course_id,rating


In [32]:
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [33]:
def load_to_dwh(recommendations, db_engine):
    recommendations.to_sql("recommendations", db_engine, if_exists="replace")

In [34]:
load_to_dwh(recommendations, db_engine)

In [38]:
def recommendations_for_user(user_id, programming_language, threshold,course_level ):
    # Join with the courses table
    query = """
    SELECT title, rating FROM recommendations
    INNER JOIN courses ON courses.course_id = recommendations.course_id
    WHERE (user_id=%(user_id)s AND programming_language=%(programming_language)s AND rating>%(threshold)s AND course_level<>%(course_level)s)
    ORDER BY rating DESC
    """
    # Add the threshold parameter
    predictions_df = pd.read_sql(query, db_engine_1, params = {"user_id": user_id, 
                                                           "threshold": threshold,
                                                            "programming_language":programming_language,
                                                              "course_level":course_level})
    return predictions_df.title.values

## ETL: Combine all the operations

In [36]:
def etl(db_engine):
    # Extract the data
    courses = extract_course_data(db_engine)
    rating = extract_rating_data(db_engine)
    
    # Clean up courses data
    courses = transform_fill_programming_language(courses)
    
    # Get the average course ratings
    avg_course_rating = transform_avg_rating(rating)
    
    # Get eligible user and course id pairs
    courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')
    
    # Calculate the recommendations
    recommendations = transform_recommendations(
    avg_course_rating,
    courses_to_recommend,
    )
    
    # Load the recommendations into the database
    load_to_dwh(recommendations, db_engine)

In [39]:
etl(db_engine)